<a href="https://colab.research.google.com/github/KaifAhmad1/code-test/blob/main/Face_Similarity_Matching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Reverse Image Search System for Defensive Forensics**

In [6]:
!pip install -q torch transformers langchain langgraph numpy pillow requests vllm langchain_community groq

In [7]:
import asyncio
import os
import torch
import numpy as np
import requests
from io import BytesIO
from PIL import Image, ImageEnhance, ImageFilter
import cv2
import matplotlib.pyplot as plt
import networkx as nx
import aiohttp
import time
from typing import Dict, List, Any, Tuple, Optional, Union
import json

# For embeddings and multimodal processing
from transformers import CLIPProcessor, CLIPModel, AutoProcessor, AutoModel
import torch.nn.functional as F

# For LLM orchestration
from groq import Groq
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_core.messages import SystemMessage, HumanMessage

# For graph-based agent orchestration
from langgraph.graph import StateGraph, END

In [9]:
import getpass

# Securely get API keys from user input
GROQ_API_KEY = getpass.getpass("Enter your GROQ API Key: ")
GOOGLE_CSE_ID = getpass.getpass("Enter your Google CSE ID: ")
GOOGLE_API_KEY = getpass.getpass("Enter your Google API Key: ")

Enter your GROQ API Key: ··········
Enter your Google CSE ID: ··········
Enter your Google API Key: ··········


In [ ]:
# Initialize global models
print("Loading models...")
clip_model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

# Load BLIP-2 for better image understanding
blip_processor = AutoProcessor.from_pretrained("Salesforce/blip2-opt-2.7b")
blip_model = AutoModel.from_pretrained("Salesforce/blip2-opt-2.7b", torch_dtype=torch.float16)

# Load DINOv2 for more robust feature extraction
dinov2_model = AutoModel.from_pretrained("facebook/dinov2-base")
dinov2_processor = AutoProcessor.from_pretrained("facebook/dinov2-base")

# Set device (use GPU if available)
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model = clip_model.to(device)
blip_model = blip_model.to(device)
dinov2_model = dinov2_model.to(device)

print(f"Models loaded on {device}")

# Initialize Groq client
client = Groq(
    api_key=GROQ_API_KEY
)

# Initialize llama_llm using chat completion setup
llama_llm = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Initialize the model.",
        }
    ],
    model="llama-3.3-70b-versatile",  # Specify the model here
    temperature=0.1
)

Loading models...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]